# Calculate the entropy

## The Credit Failure Example

In [ ]:
import sys
sys.path.append("..")
from scipy.stats import entropy
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from helpers.path_translation import translate_to_file_string

inputFile = translate_to_file_string("../data/credit_failure.csv")

In [ ]:
def calc_bin_entropy (dataframe, label="Kreditausfall"):
    """ calculates the entropy of the given dataframe based on the given label """
    numRows= dataframe.count()
    truefalse = dataframe.groupBy(label).count()
    labelvalues = csv.select(label).dropDuplicates()
    if labelvalues.count() != 2 :
        raise Exception('infalid datafram or label')
    else : 
        labelval0 = labelvalues.collect()[0][0]
        labelval1 = labelvalues.collect()[1][0]

        return entropy([truefalse.filter(f"{label} == '{labelval0}'").select("count").collect()[0]["count"] / numRows, \
                truefalse.filter (f"{label} == '{labelval1}'").select("count").collect()[0]["count"] / numRows ], base=2)

In [ ]:
spark = (SparkSession
             .builder
             .appName("Entropy")
             .getOrCreate())

csv = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ";") \
        .csv(inputFile)
csv.show()

base_entropy = calc_bin_entropy(csv) 
print (base_entropy)

In [ ]:
kopfform_rund_entropy = calc_bin_entropy(csv.filter("Kopfform == 'Rund'"))
print (kopfform_rund_entropy)
kopfform_quadratisch_entropy = calc_bin_entropy(csv.filter("Kopfform == 'Quadratisch'"))
print (kopfform_quadratisch_entropy)

In [ ]:
koerperform_rechteck = calc_bin_entropy(csv.filter("Koerperform == 'Rechteck'"))
print (koerperform_rechteck)
koerperform_oval = calc_bin_entropy(csv.filter("Koerperform == 'Oval'"))
print (koerperform_oval)

In [ ]:
koerperfarbe_weiss = calc_bin_entropy(csv.filter("Koerperfarbe == 'weiss'"))
print (koerperfarbe_weiss)
koerperfarbe_schwarz = calc_bin_entropy(csv.filter("Koerperfarbe == 'schwarz'"))
print (koerperfarbe_schwarz)

In [ ]:
spark.stop()